In [4]:
import requests
import simplejson
import unidecode
import operator
import json
import sys
from IPython.core.display import display, HTML
import pandas as pd
from collections import OrderedDict
from datetime import date
import random

pd.set_option('display.max_colwidth', -1)
display.border = 2

In [14]:
import random
file = open("palabras_3_letras.txt", "r")
#list(file)
    
def random_line(afile):
    line = next(afile)
    for num, aline in enumerate(afile, 2):
      if random.randrange(num): continue
      line = aline
    return line

print (random_line(file))


Fil



In [15]:

##########################################################################################
# PRODUCTO A BUSCAR
file = open("palabras_3_letras.txt", "r")
product = "bicicleta"
product_to_avoid = "xxx"
#file = ('bicicleta','pelota')

for product in file:
    product = (random_line(file))
    print(product) 

    min_price_list = (1,2000,10000)#,3000,8000,16000)
    max_price_list = (2000,10000,20000)#,3000,8000,16000,240000)
    # TUS BARRIOS SEPARADOS POR ESPACIO
    other_locations = "San"
    # SI EL MAXIMO VENDIDO ES 1000, y % 0.7 SOLO MUESTRA LOS QUE VENDIERON ARRIBA DE 700 
    percentage_max_sell = "0.05"
    # USED o NEW
    sell_condition = "new" #"old"
    # MINIMUM SELL FOR LOCATION
    min_sell = 1

    #file = open("mla_results_" + product + ".csv","a+")
    df_final = pd.DataFrame.from_dict([])
    
#########################################################################################

    num = 0
    for price in min_price_list:
        df_temp = ''
        df_temp = pd.DataFrame.from_dict([])
        min_price = min_price_list[num]
        max_price = max_price_list[num]
        mla_list = search_mla_return_list(product,min_price,max_price)
        #return_tuples_from_list(mla_list,product,min_price,max_price)
        max_sold = return_max_sold_from_list(mla_list,product,min_price,max_price)
        #display(HTML('<style>        table {      border-collapse: collapse;   }  table, td, th {   border: 1px solid black;            }  </style>       '))
        ranger = '$' + str(min_price) + '-$' + str(max_price)
        df = return_products_from_list_pandas(mla_list,product,sell_condition,percentage_max_sell,max_sold,min_price,max_price)
        df_temp2 = df
        #print(df)
        if (len(df) > 0):
            df['city'] = df_temp2['address'].apply(lambda x: x.get('city_name'))
            df['state'] = df_temp2['address'].apply(lambda x: x.get('state_name'))
            df['total'] = df_temp2['price'] * df_temp2['sold_quantity']
            
        for num2 in range(0,len(df)):
            df_value = df.iloc[num2]
            if ( int(df_value['total']) > 150000 ):
                #print (df_value)
                df_temp = df_temp.append(df_value, ignore_index=True)
                df_final = df_final.append(df_value, ignore_index=True)
        if ( len(df_temp)>0):        
            display(HTML(df_temp.loc[:, ['title', 'price','sold_quantity','total','state','city','thumbnail','permalink']].to_html(border=1, escape=False ,formatters=dict(thumbnail=path_to_image_html,permalink=path_to_link_html))))
            #display(HTML(df_temp.to_html(border=1, escape=False ,formatters=dict(thumbnail=path_to_image_html,permalink=path_to_link_html))))
        num = num + 1
        

Ten



NameError: name 'search_mla_return_list' is not defined

In [16]:
def path_to_image_html(path):
    '''
     This function essentially convert the image url to 
     '<img src="'+ path + '"/>' format. And one can put any
     formatting adjustments to control the height, aspect ratio, size etc.
     within as in the below example. 
    '''

    return '<img src="'+ path + '" style=max-height:80px;"/>'

def path_to_link_html(path):
    '''
     This function essentially convert the image url to 
     '<img src="'+ path + '"/>' format. And one can put any
     formatting adjustments to control the height, aspect ratio, size etc.
     within as in the below example. 
    '''

    return '<a href="'+ path + '">link</a>'


def check_ALL_list_in_title(list,string ):
# ALL NEEDS TO MATCH
    result = 1
    for item in list:
        #print ("item:",item,"str:",string)
        if item.lower() in string.lower():
            result = 1
        else:
            result = 0
    return result


def check_ANY_list_in_title(list,string ):
# ANY NEEDS TO MATCH
    result = 0
    for item in list:
        #print ("item:",item,"str:",string)
        if item.lower() in string.lower() :
            result = 1
    return result



def search_mla_return_list(product,min_price,max_price):
    #print ("Returning complete MLA list for product",product)
    link = 'https://api.mercadolibre.com/sites/MLA/search?q=\'' + product + '\'' 
    price_filter = "&price=" + str(min_price) + "-" + str(max_price)
    link2 = link + '&offset=' + str(0) + '&limit=' + str(50) + price_filter
    r = requests.get(link2)
    #print (link2)
    c = r.json()
    #print (c['results'][0])
    c_orig = c
    c_partial = c['results']
    start =  c['paging']['offset']
    end =  c['paging']['limit']
    total =  c['paging']['total'] 
    total_saved = total
    total_c = c['results']
    maximum_sell = 0
    print ("Total(limited to 1000 with no token access):",total," start:",start," end ",end)
    if total > 1000:
        total = 1000
    # TEMOPORARY
    num_new = 0
    num=0
    num_new = 0
    while ( total > 51 and end > 10 ): # and 0):  ## <--- full list instead of first 50
        start = start + end
        if ( (c['paging']['total'] - start) < end  ):
            end = c['paging']['total'] - 1 - start 
        link2 = link + '&offset=' + str(start) + '&limit=' + str(end) + price_filter
        #print ("requesting LINK:",link2)
        r = requests.get(link2)
        c = r.json()
        c_partial = 0
        #print ("Total:",c['paging']['total']," start:",start," limit ",end)
        total_c = total_c + c['results']
        total = total - end 
        num = num + 1 
    return total_c

def return_max_sold_from_list(list,product,min_price,max_price):
    #print ("----- Searching max sold...");
    product_list = product.split()
    maximum_sell = 0
    num_new = 0
    for x in list[:]:
        #print(c['site_id'], c['results']['title'],c['results']['sold_quantity'], "$",c['results']['price'],c['site_id'], c['results']['address']['city_name'])
        sold_qty = list[num_new]['sold_quantity']
        price = list[num_new]['price']
        #print ("sold qty",sold_qty,"num:",num)
        title = unidecode.unidecode(list[num_new]['title']).lower()
        if ( sold_qty > maximum_sell and check_ALL_list_in_title(product_list,title ) and not check_ALL_list_in_title(product_to_avoid.split(),title ) and ( int(price) >= int(min_price) and int(price) <= int(max_price) )):
            maximum_sell = sold_qty
            #print ("new max sell found",maximum_sell)
        num_new = num_new + 1
    #print ("....max units sold at this range: ",maximum_sell)
    return maximum_sell

def return_tuples_from_list(list,product,min_price,max_price):
    print ("----- Searching maximum units sold...")
    product_list = product.split()
    maximum_sell = 0
    num_new = 0
    for x in list[:]:
        #print(c['site_id'], c['results']['title'],c['results']['sold_quantity'], "$",c['results']['price'],c['site_id'], c['results']['address']['city_name'])
        sold_qty = list[num_new]['sold_quantity']
        price = list[num_new]['price']
        title = list[num_new]['title']
        title_list = title.split()
        title_short = title_list[0:3]
        #print ("Title short:",title_short)
        #print ("sold qty",sold_qty,"num:",num)
        title = unidecode.unidecode(list[num_new]['title']).lower()
        if ( sold_qty > maximum_sell and check_ALL_list_in_title(product_list,title ) and not check_ALL_list_in_title(product_to_avoid.split(),title ) and ( int(price) >= int(min_price) and int(price) <= int(max_price) )):
            maximum_sell = sold_qty
            #print ("Title short:",title_short) 
            #print ("new max sell found",maximum_sell)
        num_new = num_new + 1
    return maximum_sell

       
def return_products_from_list_pandas(total_c,product,condition,percentage_max_sell,max_sell,min_price,max_price):
    from operator import itemgetter
    #print ("----- Printing products from list..max units sold:",max_sell," Percentage filter at:",percentage_max_sell,"...min:",min_price," max_price:",max_price)
    num = 0
    list_new = list()
    product_list = product.split()
    df = pd.DataFrame.from_dict(total_c)
    if (len(df)>0):
        df_resu = pd.DataFrame()
        df_temp = df.loc[:, ['title', 'price','sold_quantity','address','thumbnail','permalink']].sort_values('sold_quantity',ascending=False)
        #df_temp[:20]
        df_temp['city'] = df_temp['address'].apply(lambda x: x.get('city_name'))
        df_temp['state'] = df_temp['address'].apply(lambda x: x.get('state_name'))
        df_temp2 = df_temp
        df_temp2['title'] = df_temp2['title'].str.lower()

    #for palabra in product.split(' '):
    #    df_temp2 = df_temp2[ df_temp2['title'].str.contains(palabra)]
    #    df_resu = df_resu.append(df_temp2)
    #print ("R:",df_resu,"T:",df_temp2)
    return df
        

def return_products_from_list_in_locations_pandas(total_c,product,sell_condition,locations,min_sell,min_price,max_price):
    #print ("----- Printing products from list in LOCATION..min $:",min_price," max $:",max_price)
    #sorted(total_c, key=lambda sold_quantity: sold_quantity[8])
    #sorted(total_c, key=operator.itemgetter(16) )
    df = pd.DataFrame.from_dict(total_c)
    df_resu = pd.DataFrame()
    df_temp = df.loc[:, ['title', 'price','sold_quantity','address','thumbnail','permalink']].sort_values('sold_quantity',ascending=False)
    #df_temp[:20]
    df_temp['city'] = df_temp['address'].apply(lambda x: x.get('city_name'))
    df_temp['state'] = df_temp['address'].apply(lambda x: x.get('state_name'))
    df_temp2 = df_temp
    df_temp2['title'] = df_temp2['title'].str.lower()

   
    for palabra in product.split(' '):
        df_temp2 = df_temp2[ df_temp2['title'].str.contains(palabra)]
        df_resu = df_resu.append(df_temp2)
    df_temp2 = df_resu[ df_resu['city'].str.contains(locations)]
    #print ("R:",df_resu,"T:",df_temp2)
    return df_temp2
    


In [6]:
df_final
#display(HTML(df_temp.loc[:, ['title', 'price','sold_quantity','state','city','thumbnail','permalink']].to_html(border=1, escape=False ,formatters=dict(thumbnail=path_to_image_html,permalink=path_to_link_html))))


""


In [10]:

display(HTML(df_temp.loc[:, ['title','price','sold_quantity','total','state','city','thumbnail','permalink']].sort_values(['total'],ascending=False).to_html(border=1, escape=False ,formatters=dict(thumbnail=path_to_image_html,permalink=path_to_link_html))))
            

KeyError: "None of [['title', 'price', 'sold_quantity', 'total', 'state', 'city', 'thumbnail', 'permalink']] are in the [columns]"